# Parse Analysis from the REST API

We can also get all the data we need from the REST API. This only takes into accounts filtered alleles.

## Authenticate the REST API

In [2]:
import requests
import json
from pprint import pprint

In [3]:
session = requests.Session()

url = 'http://ella-web:5000'
user_data = {"username": "testuser1", "password": "Password#123"}
session.post('{}/api/v1/users/actions/login/'.format(url), json=user_data)
print(session.cookies.get_dict())

{'AuthenticationToken': 'fb2e6d6bac61cb56de2decb663fa60fe637262b78d12b757fe317143e42b3660'}


In [19]:
# This would need to be updated for a multi sample analysis
ANALYSIS_ID = 2

### Get the Gene Panel

In [13]:
analysis_metadata = session.get('{}/api/v1/analyses/2'.format(url))
analysis_metadata = json.loads(analysis_metadata.content.decode('utf-8'))
genepanel_name, genepanel_version = analysis_metadata['genepanel']['name'], analysis_metadata['genepanel']['version']
print(genepanel_name)
print(genepanel_version)
pprint(analysis_metadata)

HBOCUTV
v01
{'date_deposited': '2020-10-19T10:46:43.419040+00:00',
 'date_requested': None,
 'genepanel': {'name': 'HBOCUTV', 'version': 'v01'},
 'id': 2,
 'interpretations': [{'analysis_id': 2,
                      'date_last_update': '2020-11-28T06:44:25.638124+00:00',
                      'finalized': True,
                      'genepanel_name': 'HBOCUTV',
                      'genepanel_version': 'v01',
                      'id': 2,
                      'status': 'Done',
                      'user': {'abbrev_name': 'H. Ibsen',
                               'active': True,
                               'first_name': 'Henrik',
                               'full_name': 'Henrik Ibsen',
                               'id': 1,
                               'last_name': 'Ibsen',
                               'user_group_name': 'testgroup01',
                               'username': 'testuser1'},
                      'user_id': 1,
                      'workflow_status': 'I

### Get the FilterConfig

In [18]:
query_url = 'api/v1/workflows/analyses/2/filterconfigs/'
response = session.get('{}/{}'.format(url, query_url))
filterconfigs = json.loads(response.content.decode('utf-8'))
#pprint(filter_configs)
filterconfig_id = None
for filterconfig in filterconfigs:
    if filterconfig['active']:
        filterconfig_id = filterconfig['id']
        break
        
assert filterconfig_id is not None

### Get The Filtered Alleles

In [8]:
# get the filtered alleles
query_url = 'api/v1/workflows/analyses/2/interpretations/2/filteredalleles/'

# Note, to get not filtered alleles use the URL (this gives a different data structure!)
# query_url = 'api/v1/workflows/analyses/2/interpretations/2/

payload = {'filterconfig': filterconfig_id}
response = session.get('{}/{}'.format(url, query_url), params=payload)
response = json.loads(response.content.decode('utf-8'))
allele_ids = response['allele_ids']
allele_ids

[3, 4, 5, 6, 7, 8]

In [25]:
query_url = 'api/v1/workflows/analyses/2/interpretations/2/alleles/'

# Allele Ids needs to be a string with comma separated values, not a list!
payload = {
            'allele_ids': ','.join([str(x) for x in allele_ids]), 
            'current': True, 
            'filterconfig_id': filterconfig_id
          }
alelle_assessments = session.get('{}/{}'.format(url, query_url), params=payload)
allele_assessments = json.loads(alelle_assessments.content.decode('utf-8'))
pprint(allele_assessments[0])

{'allele_assessment': {'allele_id': 3,
                       'analysis_id': 2,
                       'annotation_id': 3,
                       'attachment_ids': [],
                       'classification': '4',
                       'custom_annotation_id': None,
                       'date_created': '2020-10-19T10:49:44.630277+00:00',
                       'date_superceeded': None,
                       'evaluation': {'acmg': {'included': [{'code': 'PVS1',
                                                             'comment': '',
                                                             'match': None,
                                                             'op': None,
                                                             'source': 'suggested',
                                                             'uuid': '5104d287-683d-4391-a75b-5694c65f2b88'},
                                                            {'code': 'PPxPM2',
                                  

In [10]:
# Let's double check and make sure we got all the alleles
assert len(response) == len(allele_ids)

## Generate Report Data

In [28]:
report_data = {}
report_data['allele_assessments'] = allele_assessments
report_data['analysis_metadata'] = analysis_metadata
report_data['filterconfigs'] = filterconfigs

In [31]:
import json
with open('/data/analysis_rest_api_data.json', 'w') as fp:
    json.dump(report_data, fp, sort_keys=True, indent=4)